# AOC 2025 Day 10

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [2]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 10)
val input = aocClient.input()
aocClient.viewPartOne()

--- Day 10: Factory --- 
 Just across the hall, you find a large factory. Fortunately, the Elves here have plenty of time to decorate. Unfortunately, it's because the factory machines are all offline, and none of the Elves can figure out the initialization procedure. 
 The Elves do have the manual for the machines, but the section detailing the initialization procedure was eaten by a Shiba Inu . All that remains of the manual are some indicator light diagrams, button wiring schematics, and joltage requirements for each machine. 
 For example: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
 
 The manual describes one machine per line. Each line contains a single indicator light diagram in [ square brackets ] , one or more button wiring schematics in ( parentheses ) , and joltage requirements in { curly braces } . 
 To start a machine, its indicator lights must match those shown in the diagram, where . means off and # means on . The machine has the number of indicator lights shown, but its indicator lights are all initially off . 
 So, an indicator light diagram like [.##.] means that the machine has four indicator lights which are initially off and that the goal is to simultaneously configure the first light to be off, the second light to be on, the third to be on, and the fourth to be off. 
 You can toggle the state of indicator lights by pushing any of the listed buttons . Each button lists which indicator lights it toggles, where 0 means the first light, 1 means the second light, and so on. When you push a button, each listed indicator light either turns on (if it was off) or turns off (if it was on). You have to push each button an integer number of times; there's no such thing as " 0.5 presses " (nor can you push a button a negative number of times). 
 So, a button wiring schematic like (0,3,4) means that each time you push that button, the first, fourth, and fifth indicator lights would all toggle between on and off. If the indicator lights were [#.....] , pushing the button would change them to be [...##.] instead. 
 Because none of the machines are running, the joltage requirements are irrelevant and can be safely ignored. 
 You can push each button as many times as you like. However, to save on time, you will need to determine the fewest total presses required to correctly configure all indicator lights for all machines in your list. 
 There are a few ways to correctly configure the first machine: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7} 
 
 You could press the first three buttons once each, a total of 3 button presses. 
 You could press (1,3) once, (2,3) once, and (0,1) twice, a total of 4 button presses. 
 You could press all of the buttons except (1,3) once each, a total of 5 button presses. 
 
 However, the fewest button presses required is 2 . One way to do this is by pressing the last two buttons ( (0,2) and (0,1) ) once each. 
 The second machine can be configured with as few as 3 button presses: 
 [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2} 
 One way to achieve this is by pressing the last three buttons ( (0,4) , (0,1,2) , and (1,2,3,4) ) once each. 
 The third machine has a total of six indicator lights that need to be configured correctly: 
 [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5} 
 The fewest presses required to correctly configure it is 2 ; one way to do this is by pressing buttons (0,3,4) and (0,1,2,4,5) once each. 
 So, the fewest button presses required to correctly configure the indicator lights on all of the machines is 2 + 3 + 2 = 7 . 
 Analyze each machine's indicator light diagram and button wiring schematics. What is the fewest button presses required to correctly configure the indicator lights on all of the machines?

In [3]:
val lines = input.split("\n")
    .filter { it.isNotBlank() }

fun solveMachine(line: String): Int {
    // parse target state (like [.##.])
    val targetString = line.substringAfter("[").substringBefore("]")
    val numLights = targetString.length
    var targetState = 0
    targetString.forEachIndexed { index, char ->
        if (char == '#') {
            targetState = targetState or (1 shl index)
        }
    }

    // Parse buttons: (0,2) (1,3) ...
    val buttonRegex = Regex("""\(([\d,]+)\)""")
    val buttons = buttonRegex.findAll(line).map { matchResult ->
        val indices = matchResult.groupValues[1].split(",").map { it.toInt() }
        // Convert list of indices to a bitmask
        indices.fold(0) { acc, index -> acc or (1 shl index) }
    }.toList()

    return bfs(targetState, buttons)
}

fun bfs(target: Int, buttons: List<Int>): Int {
    val queue = ArrayDeque<Pair<Int, Int>>()
    queue.add(0 to 0)

    val visited = HashSet<Int>()
    visited.add(0)

    while (queue.isNotEmpty()) {
        val (current, steps) = queue.removeFirst()

        if (current == target) {
            return steps
        }

        for (btnMask in buttons) {
            val nextState = current xor btnMask

            if (visited.add(nextState)) {
                queue.add(nextState to steps + 1)
            }
        }
    }

    throw IllegalStateException("No solution found for target $target")
}

val totalPresses = lines.sumOf { line ->
    solveMachine(line)
}

aocClient.submitPartOne(totalPresses)

Your answer: 502. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [4]:
aocClient.viewPartTwo()

--- Part Two --- 
 All of the machines are starting to come online! Now, it's time to worry about the joltage requirements. 
 Each machine needs to be configured to exactly the specified joltage levels to function properly. Below the buttons on each machine is a big lever that you can use to switch the buttons from configuring the indicator lights to increasing the joltage levels. (Ignore the indicator light diagrams.) 
 The machines each have a set of numeric counters tracking its joltage levels, one counter per joltage requirement. The counters are all initially set to zero . 
 So, joltage requirements like {3,5,4,7} mean that the machine has four counters which are initially 0 and that the goal is to simultaneously configure the first counter to be 3 , the second counter to be 5 , the third to be 4 , and the fourth to be 7 . 
 The button wiring schematics are still relevant: in this new joltage configuration mode, each button now indicates which counters it affects, where 0 means the first counter, 1 means the second counter, and so on. When you push a button, each listed counter is increased by 1 . 
 So, a button wiring schematic like (1,3) means that each time you push that button, the second and fourth counters would each increase by 1 . If the current joltage levels were {0,1,2,3} , pushing the button would change them to be {0,2,2,4} . 
 You can push each button as many times as you like. However, your finger is getting sore from all the button pushing, and so you will need to determine the fewest total presses required to correctly configure each machine's joltage level counters to match the specified joltage requirements. 
 Consider again the example from before: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
 
 Configuring the first machine's counters requires a minimum of 10 button presses. One way to do this is by pressing (3) once, (1,3) three times, (2,3) three times, (0,2) once, and (0,1) twice. 
 Configuring the second machine's counters requires a minimum of 12 button presses. One way to do this is by pressing (0,2,3,4) twice, (2,3) five times, and (0,1,2) five times. 
 Configuring the third machine's counters requires a minimum of 11 button presses. One way to do this is by pressing (0,1,2,3,4) five times, (0,1,2,4,5) five times, and (1,2) once. 
 So, the fewest button presses required to correctly configure the joltage level counters on all of the machines is 10 + 12 + 11 = 33 . 
 Analyze each machine's joltage requirements and button wiring schematics. What is the fewest button presses required to correctly configure the joltage level counters on all of the machines?

In [3]:
fun solvePart2(input: String): Long {
    val machines = input.lines().filter { it.isNotBlank() }

    return machines.parallelStream().mapToLong { machineStr ->
        solveMachineP2(machineStr)
    }.sum()
}

fun solveMachineP2(block: String): Long {
    // Parse Targets {3, 5, ...}
    val targetSection = block.substringAfter("{").substringBefore("}")
    val targets = targetSection.split(",").map { it.trim().toLong() }.toLongArray()

    // Parse Buttons (0,2), (1)...
    // We build a matrix: rows = counters, cols = buttons
    // value[r][c] = 1 if button c affects counter r, else 0
    val buttonMatches = Regex("""\(([\d,]+)\)""").findAll(block).toList()
    val numButtons = buttonMatches.size
    val numCounters = targets.size

    // coefficients[row][col]
    val coefficients = Array(numCounters) { IntArray(numButtons) }

    buttonMatches.forEachIndexed { btnIdx, match ->
        val affectedCounters = match.groupValues[1].split(",").map { it.toInt() }
        for (counterIdx in affectedCounters) {
            coefficients[counterIdx][btnIdx] = 1
        }
    }

    val solver = EquationSolver(coefficients, targets)
    return solver.findMinPresses() ?: 0L
}

class EquationSolver(
    private val matrix: Array<IntArray>, // [row][col]
    private val targets: LongArray
) {
    private val numVars = matrix[0].size
    private val numEqs = matrix.size
    private var minTotalPresses = Long.MAX_VALUE

    // Current assignment of variables (button presses). -1L means unset.
    private val assignment = LongArray(numVars) { -1L }

    fun findMinPresses(): Long? {
        search(0L)
        return if (minTotalPresses == Long.MAX_VALUE) null else minTotalPresses
    }

    private fun search(currentPresses: Long) {
        // Pruning: if we already exceeded the best solution found so far
        if (currentPresses >= minTotalPresses) return

        // Find the most constrained equation (row with fewest unset variables)
        var bestRow = -1
        var minUnsetCount = Int.MAX_VALUE

        // We also track the current residual for validity checks
        val currentSums = LongArray(numEqs)

        // Calculate current sums based on assignment
        for (r in 0 until numEqs) {
            var sum = 0L
            var unset = 0
            for (c in 0 until numVars) {
                if (matrix[r][c] > 0) {
                    if (assignment[c] != -1L) {
                        sum += assignment[c] * matrix[r][c]
                    } else {
                        unset++
                    }
                }
            }
            currentSums[r] = sum

            // Check for contradiction early
            if (sum > targets[r]) return
            // If no unset variables, sum must match target exactly
            if (unset == 0 && sum != targets[r]) return

            // Heuristic: Pick row with smallest non-zero unset variables
            if (unset > 0 && unset < minUnsetCount) {
                minUnsetCount = unset
                bestRow = r
            }
        }

        // Base case: All equations satisfied
        if (bestRow == -1) {
            minTotalPresses = min(minTotalPresses, currentPresses)
            return
        }

        // Identify the variables in this row that are not yet set
        val unsetVars = ArrayList<Int>()
        for (c in 0 until numVars) {
            if (matrix[bestRow][c] > 0 && assignment[c] == -1L) {
                unsetVars.add(c)
            }
        }

        // If there's only 1 unknown, it's deterministic
        if (unsetVars.size == 1) {
            val v = unsetVars[0]
            val coeff = matrix[bestRow][v]
            val needed = targets[bestRow] - currentSums[bestRow]

            if (needed >= 0 && needed % coeff == 0L) {
                val presses = needed / coeff
                assignment[v] = presses
                search(currentPresses + presses)
                assignment[v] = -1L // Backtrack
            }
            return
        }

        // If > 1 unknowns, we branch.
        val branchVar = unsetVars[0]
        val coeff = matrix[bestRow][branchVar]

        // Maximum possible value for this variable given the target limit
        val remainingSpace = targets[bestRow] - currentSums[bestRow]
        val maxVal = remainingSpace / coeff

        for (k in 0L..maxVal) {
            assignment[branchVar] = k
            search(currentPresses + k)
            assignment[branchVar] = -1L
        }
    }
}

val answerPt2 = solvePart2(input)

aocClient.submitPartTwo(answerPt2)

Your answer: 21467. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 10! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .